<a href="https://colab.research.google.com/github/gnipka/Model-SCINet/blob/main/%D0%92%D0%9A%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка данных

In [7]:
import pandas as pd

data = pd.read_csv('sample_data/data_1.csv', parse_dates=['time'], sep=";", encoding='cp1251', low_memory=False)

In [8]:
features_considered = ['Defects.Roll10Sqm.DefMap1', 
                       'OPC_V_voronka_K02', 'OPC_V_shnek_K02',
                       'OPC_KN_T_ist_Z1_K02', 'OPC_KN_T_ist_Z2_K02', 'OPC_KN_T_ist_Z3_K02']

features = data[features_considered]
features.index = data['time']

In [9]:
features = features.interpolate()

In [10]:
features = features[(features.index > '2020-10-21') & (features.index < '2020-10-29')]

In [11]:
features = features.rename(columns={'Defects.Roll10Sqm.DefMap1' : 'OT'})
features.index.names = ['date']
features

,OT,OPC_V_voronka_K02,OPC_V_shnek_K02,OPC_KN_T_ist_Z1_K02,OPC_KN_T_ist_Z2_K02,OPC_KN_T_ist_Z3_K02
date,,,,,,
2020-10-21 00:00:10,22.034,12.146000,93.36,65.700000,136.100000,140.600000
2020-10-21 00:00:20,21.848,12.122667,94.08,65.700000,136.066667,140.600000
2020-10-21 00:00:30,21.888,12.099333,94.80,65.700000,136.033333,140.600000
2020-10-21 00:00:40,21.751,12.076000,95.52,65.700000,136.000000,140.600000
2020-10-21 00:00:50,21.959,12.074000,95.07,65.700000,136.000000,140.600000
...,...,...,...,...,...,...
2020-10-28 23:59:12,18.492,16.087333,141.89,63.300000,132.366667,137.966667
2020-10-28 23:59:22,18.503,16.098000,141.75,63.300000,132.400000,138.000000
2020-10-28 23:59:32,18.456,14.804000,130.64,63.266667,132.400000,138.066667


In [13]:
features.to_csv('data.csv', ',', encoding='utf-8')

# Подключение к диску

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.insert(0,'/content/drive/My Drive/SCINet')

# Подключение необходимых версий библиотек для работы SCINet

In [ ]:
!pip install numpy==1.19.4
!pip install pandas
!pip install scikit_learn
!pip install tensorboard
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

# Настройка модели

In [5]:
import argparse
import os
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from experiments.exp_ETTh import Exp_ETTh

class Args:
  
  model = 'SCINet'

  ### -------  dataset settings --------------
  data = 'data'
  root_path = 'sample_data/'
  data_path = 'data.csv'
  features = 'M' # может быть 'S' или 'M'
  target = 'OT' # наименование столбца в датасете
  freq = 's' # частота для кодирования времени
  checkpoints = 'exp/data_checkpoints/' # расположение контрольных точек
  inverse = False # денормализация выходных данных
  embed = 'timeF' 

  ### -------  device settings --------------
  use_gpu = True # использование gpu
  gpu = 0
  use_multi_gpu = False 
  devices = '0'

  ### -------  input/output length settings --------------  
  seq_len = 96 # look back window, начальная длина SCINet encoder
  label_len = 48 # начальная длина Informer decoder
  pred_len = 48 # длина поледовательности прогнозирования
  concat_len = 0
  single_step = 0
  single_step_output_One = 0
  lastWeight = 1.0

  ### -------  training settings --------------  
  cols = ""
  num_workers = 0 # количество работников-загрузчиков данных ??
  itr = 0
  train_epochs = 100 # количество эпох для тренировки
  batch_size = 32
  patience = 5
  lr = 0.0001
  loss = "mae" # оценочная функция
  lradj = 1
  use_amp = False
  save = True # сохранение выходных результатов
  model_name = 'SCINet'
  resume = False
  evaluate = False

  ### -------  model settings --------------  
  hidden_size = 1
  INN = 1
  kernel = 5 # 3,5,7
  dilation = 1
  window_size = 12
  dropout = 0.5
  positionalEcoding = False
  groups = 1
  levels = 3
  stacks = 1
  num_decoder_layer = 1
  RIN = False
  decompose = False

  def __init__(self, data, features, seq_len, label_len, pred_len, hidden_size, stacks, levels, lr, batch_size, dropout, model_name):
    self.data = data
    self.features = features
    self.seq_len = seq_len
    self.label_len = label_len
    self.pred_len = pred_len
    self.hidden_size = hidden_size
    self.stacks = stacks
    self.levels = levels
    self.lr = lr
    self.batch_size = batch_size
    self.dropout = dropout
    self.model_name = model_name

args = Args('data', 'S', 64, 24, 24, 8, 1, 3, 0.007, 64, 0.25, 'data_M_I48_O24_lr3e-3_bs8_dp0.5_h4_s1l3')

#if args.data in data_parser.keys():
 #   data_info = data_parser[args.data]
  #  args.data_path = data_info['data']
   # args.target = data_info['T']
    #args.enc_in, args.dec_in, args.c_out = data_info[args.features]

args.detail_freq = args.freq
args.freq = args.freq[-1:]

torch.manual_seed(4321)  # reproducible
torch.cuda.manual_seed_all(4321)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True  # Can change it to False --> default: False
torch.backends.cudnn.enabled = True

Exp = Exp_ETTh

mae_ = []
maes_ = []
mse_ = []
mses_ = []

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))

force_cudnn_initialization()
if args.evaluate:
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_lr{}_bs{}_hid{}_s{}_l{}_dp{}_inv{}_itr0'.format(args.model,args.data, args.features, args.seq_len, args.label_len, args.pred_len,args.lr,args.batch_size,args.hidden_size,args.stacks, args.levels,args.dropout,args.inverse)
    exp = Exp(args)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    mae, maes, mse, mses = exp.test(setting, evaluate=True)
    print('Final mean normed mse:{:.4f},mae:{:.4f},denormed mse:{:.4f},mae:{:.4f}'.format(mse, mae, mses, maes))
else:
    if args.itr:
        for ii in range(args.itr):
            # setting record of experiments
            setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_lr{}_bs{}_hid{}_s{}_l{}_dp{}_inv{}_itr{}'.format(args.model,args.data, args.features, args.seq_len, args.label_len, args.pred_len,args.lr,args.batch_size,args.hidden_size,args.stacks, args.levels,args.dropout,args.inverse,ii)

            exp = Exp(args)  # set experiments
            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
            exp.train(setting)

            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            mae, maes, mse, mses = exp.test(setting)
            mae_.append(mae)
            mse_.append(mse)
            maes_.append(maes)
            mses_.append(mses)

            torch.cuda.empty_cache()
        print('Final mean normed mse:{:.4f}, std mse:{:.4f}, mae:{:.4f}, std mae:{:.4f}'.format(np.mean(mse_), np.std(mse_), np.mean(mae_),np.std(mae_)))
        print('Final mean denormed mse:{:.4f}, std mse:{:.4f}, mae:{:.4f}, std mae:{:.4f}'.format(np.mean(mses_),np.std(mses_), np.mean(maes_), np.std(maes_)))
        print('Final min normed mse:{:.4f}, mae:{:.4f}'.format(min(mse_), min(mae_)))
        print('Final min denormed mse:{:.4f}, mae:{:.4f}'.format(min(mses_), min(maes_)))
    else:
        setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_lr{}_bs{}_hid{}_s{}_l{}_dp{}_inv{}_itr0'.format(args.model,args.data, args.features, args.seq_len, args.label_len, args.pred_len,args.lr,args.batch_size,args.hidden_size,args.stacks, args.levels,args.dropout,args.inverse)
        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        mae, maes, mse, mses = exp.test(setting)
        print('Final mean normed mse:{:.4f},mae:{:.4f},denormed mse:{:.4f},mae:{:.4f}'.format(mse, mae, mses, maes))

SCINet(
  (blocks1): EncoderTree(
    (SCINet_Tree): SCINet_Tree(
      (workingblock): LevelSCINet(
        (interact): InteractorLevel(
          (level): Interactor(
            (split): Splitting()
            (phi): Sequential(
              (0): ReplicationPad1d((3, 3))
              (1): Conv1d(1, 8, kernel_size=(5,), stride=(1,))
              (2): LeakyReLU(negative_slope=0.01, inplace=True)
              (3): Dropout(p=0.25, inplace=False)
              (4): Conv1d(8, 1, kernel_size=(3,), stride=(1,))
              (5): Tanh()
            )
            (psi): Sequential(
              (0): ReplicationPad1d((3, 3))
              (1): Conv1d(1, 8, kernel_size=(5,), stride=(1,))
              (2): LeakyReLU(negative_slope=0.01, inplace=True)
              (3): Dropout(p=0.25, inplace=False)
              (4): Conv1d(8, 1, kernel_size=(3,), stride=(1,))
              (5): Tanh()
            )
            (P): Sequential(
              (0): ReplicationPad1d((3, 3))
              (

RuntimeError: ignored

	iters: 100, epoch: 1 | loss: 0.0373032
	speed: 0.0583s/iter; left time: 4402.2776s
	iters: 200, epoch: 1 | loss: 0.0317414
	speed: 0.0421s/iter; left time: 3172.7575s
	iters: 300, epoch: 1 | loss: 0.0355631
	speed: 0.0421s/iter; left time: 3166.6614s
	iters: 400, epoch: 1 | loss: 0.0368229
	speed: 0.0426s/iter; left time: 3205.7197s
	iters: 500, epoch: 1 | loss: 0.0651039
	speed: 0.0419s/iter; left time: 3144.4529s
	iters: 600, epoch: 1 | loss: 0.0201411
	speed: 0.0411s/iter; left time: 3081.7995s
	iters: 700, epoch: 1 | loss: 0.0550040
	speed: 0.0422s/iter; left time: 3160.0983s
Epoch: 1 cost time: 33.397711753845215
--------start to validate-----------
normed mse:0.0239, mae:0.0544, rmse:0.1545, mape:1.6460, mspe:3529.7668, corr:0.8973
denormed mse:18.8532, mae:1.5295, rmse:4.3420, mape:inf, mspe:inf, corr:0.8973
--------start to test-----------
normed mse:0.8043, mae:0.1313, rmse:0.8968, mape:0.2654, mspe:5.4201, corr:0.7603
denormed mse:635.1712, mae:3.6899, rmse:25.2026, mape:inf